# Project 2: Covid

In [3]:
# Import
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pandas_profiling import ProfileReport
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Literature

...

## 2. EDA, feature cleaning and engineering

### 2.1. Load data

TODO: expliquer d'où provient data_selection

### 2.2 EDA